<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [3]:
!pip install ta
!pip install pandas
!pip install plotly

--2021-12-21 07:03:22--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77AYD7DUZBJL2HZ6KM3LBYF6GG
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1011 [text/plain]
Saving to: ‘helpers.py?token=ANF77AYD7DUZBJL2HZ6KM3LBYF6GG’

helpers.py?token=AN 100%[===================>]    1011  --.-KB/s    in 0s      

2021-12-21 07:03:22 (41.8 MB/s) - ‘helpers.py?token=ANF77AYD7DUZBJL2HZ6KM3LBYF6GG’ saved [1011/1011]



In [8]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77AYD7DUZBJL2HZ6KM3LBYF6GG -O helpers.py

--2021-12-21 07:06:15--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77AYD7DUZBJL2HZ6KM3LBYF6GG
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1011 [text/plain]
Saving to: ‘helpers.py’

helpers.py          100%[===================>]    1011  --.-KB/s    in 0s      

2021-12-21 07:06:15 (35.3 MB/s) - ‘helpers.py’ saved [1011/1011]



Import Libs

In [10]:
import helpers
import pandas as pd
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots
import numpy as np

In [101]:
def get_previous_candles(df, index, n, include_index=False):
    '''
    Returns previous n candles from the given index in the DataFrame

    Parameters:
    df (DataFrame): DataFrame from which to return the previous candles
    index (DataFrame Index): DataFrame Index from which to return the previous candles
    n (int): Number of previous candles to return from index
    include_index (bool): If current index should be included in returned DataFrame

    Returns:
    DataFrame: Pandas dataframe with the previous n candles
    '''
    loc = df.index.get_loc(index)
    fromIdx = loc-n
    toIdx = loc+1 if include_index else loc
    return df.iloc[fromIdx:toIdx]

def get_next_candles(df, index, n):
    '''
    Returns next n candles from the given index in the DataFrame

    Parameters:
    df (DataFrame): DataFrame from which to return the next candles
    index (DataFrame Index): DataFrame Index from which to return the next candles
    n (int): Number of next candles to return from index

    Returns:
    DataFrame: Pandas dataframe with the next n candles
    '''
    loc = df.index.get_loc(index)
    return df.iloc[loc+1:loc+1+n]

## Load 15 min and 60 min stock data and calculate RSI

In [18]:
file_15min = '/content/stock_data/RELIANCE21DECFUT-HIST-15M.csv'
file_60min = '/content/stock_data/ACC-HIST-60M.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-20 15:00:00+05:30,2272.75,2282.40,2272.50,2278.80,501750,42.878483
2021-12-20 15:15:00+05:30,2278.95,2283.00,2278.25,2279.60,542500,43.527920
2021-12-21 09:15:00+05:30,2310.00,2311.35,2296.75,2301.45,791000,57.680211
2021-12-21 09:30:00+05:30,2301.25,2320.00,2296.40,2317.80,685500,64.790726
2021-12-21 09:45:00+05:30,2317.05,2328.65,2312.70,2323.55,593750,66.897205


In [ ]:
#df_60min = pd.read_csv(file_60min, parse_dates=['Date'], index_col=['Date'])
#rsi = RSIIndicator(df_60min['Close']).rsi()
#df_60min = df_60min.assign(rsi=rsi.values)
#df_60min.tail()

## Set Strategy Parameters

In [146]:
window_start = '2021-12-01 00:00:00'
window_end = '2021-12-18 00:00:00'
rsi_15min = 70
rsi_60min = 70
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [143]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-09 09:30:00+05:30,2432.00,2464.00,2431.4,2463.80,1201250,80.812575
2021-12-09 10:00:00+05:30,2456.50,2469.80,2455.0,2459.00,675750,73.200425
2021-12-09 10:30:00+05:30,2459.55,2465.80,2458.0,2463.90,222500,74.690700
2021-12-09 12:30:00+05:30,2465.55,2474.40,2464.5,2470.50,433750,71.950087
2021-12-09 12:45:00+05:30,2470.50,2481.05,2470.5,2478.15,478000,75.419213



### Step 4: Volume check

In [144]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = get_previous_candles(curr_window_df, index, back_candles)
  # print('Current candle:', index)
  # print('Back candles:', back_candles_df)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-12-09 09:30:00+05:30,2432.00,2464.00,2431.40,2463.80,1201250.0,80.812575,335050.0
2021-12-09 12:30:00+05:30,2465.55,2474.40,2464.50,2470.50,433750.0,71.950087,177050.0
2021-12-09 12:45:00+05:30,2470.50,2481.05,2470.50,2478.15,478000.0,75.419213,193900.0
2021-12-10 15:00:00+05:30,2459.00,2471.00,2458.85,2470.20,505750.0,70.176933,189800.0


### Step 5: Hourly candle RSI check

In [147]:
temp_df = pd.DataFrame(columns=df.columns)
hourly_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'
hourly_df.index.name = 'Date'
for index, row in df.iterrows():
  prev_candles = get_previous_candles(df_15min, index, 3, True)
  mean_rsi = prev_candles['rsi'].mean()
  print('mean_rsi:', mean_rsi)
  if(mean_rsi > rsi_60min):
    # This 15min candle is eligible for signal
    print('Hourly candle RSI is greater than 50..')
    temp_df.loc[index] = row

df = temp_df
df

mean_rsi: 65.18757216208002
mean_rsi: 68.74393503146901
mean_rsi: 71.21608064866878
Hourly candle RSI is greater than 50..
mean_rsi: 62.57572260517209


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-09 12:45:00+05:30,2470.5,2481.05,2470.5,2478.15,478000.0,75.419213


### Step 6: Stock movement check

In [148]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, 5)
  min_low = prev_candles['Low'].min()
  daily_movement = row['High'] - min_low
  tanaji_pct = (daily_movement/min_low)*100
  print('daily_movement:', daily_movement,'tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

df = temp_df
temp_df

daily_movement: 32.80000000000018 tanaji_pct: 1.3397324619626338


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-09 12:45:00+05:30,2470.5,2481.05,2470.5,2478.15,478000.0,75.419213


## **Final Signal**

In [149]:
for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-12-09 12:45:00+05:30 , rsi: 75.41921330456711 Entry Price 2481.05 Stop Loss: 2462.7


In [141]:
import plotly.graph_objects as go
import plotly.offline as py

for index, row in df.iterrows():
  prev_candles = get_previous_candles(df_15min, index, 6, True)
  next_candles = get_next_candles(df_15min, index, 3)
  candles = pd.concat([prev_candles, next_candles])
  # candles.loc[index] = row

  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  fig = make_subplots(rows=2, cols=1, shared_xaxes=False,
               vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume & RSI'), 
               row_width=[0.5, 0.7],
               specs=[[{"secondary_y": False}], [{"secondary_y": True}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=6,y=candles['High'][6],
                     text='Signal')

  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.1]*df.size)),
           secondary_y=True,
          row=2, col=1)
  
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='rgb(55, 55, 109)'),
    row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=10,y0=70, y1=70, 
                line=dict(color='Green'),
                row=2,col=1)


  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=900,
    height=750,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  